In [18]:
import pandas as pd
import openpyxl
import geopandas as gp
import geopy
import os
from pathlib import Path
import time
import random

In [70]:
df = pd.read_excel(Path(os.getcwd(), "Assets", "Data", "locations.xlsx"))
df.head()

,Brand,Name,Address,Open_time,Close_time
0,FEBO,FEBO Reguliersbreestraat Amsterdam,Reguliersbreestraat 38 1017 CN Amsterdam,"{“Maandag”: “11:00”, “Dinsdag”: “11:00”, “Woen...","{“Maandag”: “3:00”, “Dinsdag”: “3:00”, “Woensd..."
1,FEBO,FEBO Hoofddorp,Marktplein 220 2132 CW Hoofddorp,NaN,NaN
2,FEBO,FEBO Diagonaal Almere,De Diagonaal 26 1315 XE Almere,NaN,NaN
3,FEBO,FEBO Almere-Buiten,Noordeinde 176 1334 BR Almere,NaN,NaN
4,FEBO,FEBO Amersfoort,Langestraat 84 3811 AK Amersfoort,NaN,NaN


In [71]:
# Geocoding
geolocator = geopy.geocoders.Nominatim(user_agent="test")
location = geolocator.geocode(df.Address.iloc[0])
print(location.raw)

{'place_id': 144018272, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 1719407868, 'lat': '52.3663683', 'lon': '4.8951869', 'class': 'amenity', 'type': 'fast_food', 'place_rank': 30, 'importance': 8.268177309829888e-05, 'addresstype': 'amenity', 'name': 'FEBO', 'display_name': 'FEBO, 38, Reguliersbreestraat, Grachtengordel, Centrum, Amsterdam, Noord-Holland, Nederland, 1017 CN, Nederland', 'boundingbox': ['52.3663183', '52.3664183', '4.8951369', '4.8952369']}


In [73]:
proj_name = "FoodPrisonFinder"

def get_coords(row, geo_agent):
    time.sleep(random.random() * 2)
    try:
        location = geo_agent.geocode(row["Address"]).raw
        row["y"] = location['lat']
        row["x"] = location['lon']
    except: # UnboundLocalError or AttributeError:
        print(row["Address"])
        row["y"] = 0 
        row["x"] = 0 #TODO: fix to remove 

    return row

def address_to_geojson(df: pd.DataFrame, save_name: str = "", in_coord_sys: str = "EPSG:4326", out_coord_sys: str = "EPSG:4326"):
    # Make geo agent and then apply geocode fn
    geo_agent = geopy.geocoders.Nominatim(user_agent = proj_name)
    df = df.apply(lambda x: get_coords(x, geo_agent), axis = 1)
    # Convert to geodataframe and then export in GeoJson format
    gdf = gp.GeoDataFrame(
    df, geometry=gp.points_from_xy(df.x, df.y), crs = in_coord_sys)
    gdf.to_file(save_name, driver="GeoJSON") 
    print("File saved.")
    return gdf

gdf = address_to_geojson(df, "locations.json")


Arenaboulevard 135 1101 EJ Amsterdam 
Buikslotermeerplein 54 1025 EW Amsterdam 
Rijksweg 2 5314 LJ Bruchem 
Tijnmuiden 53 unit A1a 1046 AK Amsterdam 
ein 45E Amsterdam 
File saved.


In [33]:
gdf.to_file("locations", driver="GeoJSON") 

In [35]:
import datetime as dt 
days = {0:"Maandag", 1:"Dinsdag", 2:"Woensdag", 3:"Donderdag", 4:"Vrijdag", 5:"Zaterdag", 6:"Zondag"}
days[dt.datetime.today().weekday()]

'Vrijdag'

In [74]:
now = dt.datetime.now() - dt.timedelta(hours=5)
now = now.time()
closing = dt.datetime.strptime("00:00", "%H:%M").time()
opening = dt.datetime.strptime("11:00", "%H:%M").time()
if now < closing:
    print("Still open!")
elif now > closing:
    print("Closed champ")

Closed champ


In [61]:
time_left = dt.datetime.combine(dt.date.min, closing) - dt.datetime.combine(dt.date.min, now)
# time_left += dt.timedelta(days=1)
print(now, time_left)

19:01:41.278872 3:58:18.721128


In [69]:
if time_left <= dt.timedelta(minutes = 20):
    print("y")
elif dt.timedelta(hours = 1) > time_left > dt.timedelta(minutes = 20):
    print("yy")
elif time_left > dt.timedelta(hours = 1):
    print("yyy")

yyy
